In [7]:
import pandas as pd
from datetime import datetime


def preprocessing():
    # 문자형
    raw['BROD_DT'] = raw['BROD_DT'].astype(str)
    raw['BFMT_NO'] = raw['BFMT_NO'].astype(str)
    raw['SLITM_CD'] = raw['SLITM_CD'].astype(str)
    raw['ORD_NO'] = raw['ORD_NO'].astype(str)
    raw['CUST_NO'] = raw['CUST_NO'].astype(str)
    
    # 정수형
    raw['INSM_MTHS'] = pd.to_numeric(raw['INSM_MTHS'], errors='coerce').astype('Int64')
    
    # 카테고리형
    raw['ITEM_GBCD'] = raw['ITEM_GBCD'].astype('category')
    raw['ITEM_GBNM'] = raw['ITEM_GBNM'].astype('category')
    raw['INTG_ITEM_GBCD'] = raw['INTG_ITEM_GBCD'].astype('category')
    raw['INTG_ITEM_GBNM'] = raw['INTG_ITEM_GBNM'].astype('category')
    raw['LAST_ORD_STAT_GBCD'] = raw['LAST_ORD_STAT_GBCD'].astype('category')
    raw['LAST_ORD_STAT_GBNM'] = raw['LAST_ORD_STAT_GBNM'].astype('category')
    raw['ITEM_MDA_GBCD'] = raw['ITEM_MDA_GBCD'].astype('category')
    raw['ITEM_MDA_GBNM'] = raw['ITEM_MDA_GBNM'].astype('category')
    raw['ACPT_CH_GBCD'] = raw['ACPT_CH_GBCD'].astype('category')
    raw['ACPT_CH_GBNM'] = raw['ACPT_CH_GBNM'].astype('category')
    raw['LAST_STLM_STAT_GBCD'] = raw['LAST_STLM_STAT_GBCD'].astype('category')
    raw['LAST_STLM_STAT_GBNM'] = raw['LAST_STLM_STAT_GBNM'].astype('category')
    raw['PAY_WAY_GBCD'] = raw['PAY_WAY_GBCD'].astype('category')
    raw['PAY_WAY_GBNM'] = raw['PAY_WAY_GBNM'].astype('category')
    raw['PAY_WAY_GBCD'] = raw['PAY_WAY_GBCD'].astype('category')
    raw['PAY_WAY_GBCD'] = raw['PAY_WAY_GBCD'].astype('category')
    raw['PAY_WAY_GBCD'] = raw['PAY_WAY_GBCD'].astype('category')
    
    # 날짜형
    raw['BROD_STRT_DTM'] = pd.to_datetime(raw['BROD_STRT_DTM'], errors='coerce')
    raw['BROD_END_DTM'] = pd.to_datetime(raw['BROD_END_DTM'], errors='coerce')
    raw['PTC_ORD_DTM'] = pd.to_datetime(raw['PTC_ORD_DTM'], errors='coerce')
    raw['ORD_STAT_PROC_DTM'] = pd.to_datetime(raw['ORD_STAT_PROC_DTM'], errors='coerce')


def create_rfm():
    # RFM 분석 : CUST_NO
    # R : 최근에 구매했는지 (날짜) PTC_ORD_DTM
    # F : 자주 구매했는지 (횟수) ORD_NO
    # M : 얼마나 많이 구매했는지 (금액) SELL_UPRC
    
    # 1. R : 최종 구매 경과일
    df_r = raw.sort_values(by=['CUST_NO', 'PTC_ORD_DTM'], ascending=False)
    df_r = df_r.drop_duplicates(subset=['CUST_NO'])
    df_r = df_r[['CUST_NO', 'PTC_ORD_DTM']]
    
    df_r['LAST_DAYS'] = (now - df_r['PTC_ORD_DTM']).dt.days
    
    # 2. F : 최종 구매 경과일
    df_f = raw.drop_duplicates(subset=['CUST_NO', 'ORD_NO'])
    
    df_f = df_f.pivot_table(
        index='CUST_NO'
        , values='ORD_NO'
        , aggfunc='count'
    )
    
    df_f = df_f.reset_index()
    df_f.columns = ['CUST_NO', 'FREQUENCY']
    
    # 3. M : 고객번호별 주문 총 누계금액
    df_m = raw.pivot_table(
        index='CUST_NO'
        , values='SELL_UPRC'
        , aggfunc='sum'
    )
    
    df_m = df_m.reset_index()
    df_m.columns = ['CUST_NO', 'SUM_AMT']
    
    # 4. RFM 
    df_rfm = pd.merge(df_r, df_f, how='inner', on='CUST_NO') 
    df_rfm = pd.merge(df_rfm, df_m, how='inner', on='CUST_NO')

    return df_rfm
    

def analyze_rfm(df_rfm):
    print(df_rfm)


def main():
    now = datetime.now()
    raw = pd.read_csv('./BFMT_ORD_250413-250419.csv', encoding='euc-kr')

    preprocessing()
    df_rfm = create_rfm()
    analyze_rfm(df_rfm)

# main() 함수 실행
if __name__ == "__main__":
    main()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3568\3790360964.py:90: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv('./BFMT_ORD_250413-250419.csv', encoding='euc-kr')


            CUST_NO         PTC_ORD_DTM  LAST_DAYS  FREQUENCY   SUM_AMT
0      202504170101 2025-04-19 23:53:21          3          1    438000
1      202504167929 2025-04-19 17:12:39          3          1  27588000
2      202504167913 2025-04-19 16:48:43          3          1    239700
3      202504167911 2025-04-19 16:45:08          3          1    239700
4      202504167909 2025-04-19 16:43:51          3          1    239700
...             ...                 ...        ...        ...       ...
33564  200101021552 2025-04-13 14:16:27         10          1    239700
33565  200101021426 2025-04-16 07:54:15          7          1     29900
33566  200101021010 2025-04-14 05:37:28          9          1     49000
33567  200101018308 2025-04-16 10:09:10          7          1     99800
33568  200101006953 2025-04-18 06:04:27          5          1     49900

[33569 rows x 5 columns]
